In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definição da arquitetura LeNet-5
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # 1 canal (imagem em grayscale)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # Ajuste para MNIST (28x28)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes (dígitos de 0 a 9)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Preparação do dataset MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalização para [-1, 1]
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inicialização do modelo, função de perda e otimizador
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função de treino
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Backward pass e atualização dos pesos
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(train_loader)

# Função de teste
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)
            test_loss += criterion(output, target).item()

            # Acertos
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

# Loop de treinamento
epochs = 10
for epoch in range(1, epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, accuracy = test(model, test_loader, criterion, device)

    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}, Accuracy = {accuracy:.2f}%")


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1: Train Loss = 0.2647, Test Loss = 0.0826, Accuracy = 97.44%
Epoch 2: Train Loss = 0.0758, Test Loss = 0.0543, Accuracy = 98.22%
Epoch 3: Train Loss = 0.0544, Test Loss = 0.0406, Accuracy = 98.58%
Epoch 4: Train Loss = 0.0446, Test Loss = 0.0372, Accuracy = 98.71%
Epoch 5: Train Loss = 0.0379, Test Loss = 0.0422, Accuracy = 98.66%
Epoch 6: Train Loss = 0.0315, Test Loss = 0.0401, Accuracy = 98.70%
Epoch 7: Train Loss = 0.0274, Test Loss = 0.0385, Accuracy = 98.78%
Epoch 8: Train Loss = 0.0229, Test Loss = 0.0406, Accuracy = 98.91%
Epoch 9: Train Loss = 0.0212, Test Loss = 0.0350, Accuracy = 98.97%
Epoch 10: Train Loss = 0.0197, Test Loss = 0.0390, Accuracy = 98.79%


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definição da arquitetura LeNet-5 (mesma anterior)
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Função para calcular a entropia dos pesos
def entropy_regularization(model):
    entropy = 0.0
    for param in model.parameters():
        if param.requires_grad:
            # Normaliza os pesos para criar uma distribuição discreta
            weights = param.view(-1)
            prob = torch.abs(weights) / torch.sum(torch.abs(weights))
            prob = prob + 1e-10  # Evitar log(0)
            entropy -= torch.sum(prob * torch.log(prob))
    return entropy

# Configurações e dados
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loop de treinamento com regularização por entropia
epochs = 10
lambda_entropy = 0.01  # Peso do termo de regularização

for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Regularização por entropia
        entropy_loss = entropy_regularization(model)
        total_loss = loss + lambda_entropy * entropy_loss

        # Backward pass e otimização
        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    # Avaliação no conjunto de teste
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Epoch {epoch}: Train Loss = {running_loss / len(train_loader):.4f}, "
          f"Test Loss = {test_loss:.4f}, Accuracy = {accuracy:.2f}%")


Epoch 1: Train Loss = 0.6847, Test Loss = 0.0880, Accuracy = 97.42%
Epoch 2: Train Loss = 0.4374, Test Loss = 0.0482, Accuracy = 98.46%
Epoch 3: Train Loss = 0.3980, Test Loss = 0.0367, Accuracy = 98.85%
Epoch 4: Train Loss = 0.3807, Test Loss = 0.0393, Accuracy = 98.74%
Epoch 5: Train Loss = 0.3718, Test Loss = 0.0376, Accuracy = 98.69%
Epoch 6: Train Loss = 0.3625, Test Loss = 0.0341, Accuracy = 98.84%
Epoch 7: Train Loss = 0.3529, Test Loss = 0.0330, Accuracy = 98.95%
Epoch 8: Train Loss = 0.3440, Test Loss = 0.0353, Accuracy = 98.79%
Epoch 9: Train Loss = 0.3546, Test Loss = 0.0320, Accuracy = 99.06%
Epoch 10: Train Loss = 0.3323, Test Loss = 0.0332, Accuracy = 98.95%


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definição da arquitetura LeNet-5
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Função para calcular a entropia dos pesos
def entropy_regularization(model):
    total_entropy = 0.0
    for param in model.parameters():
        if param.requires_grad:
            # Normaliza os pesos para criar uma distribuição discreta
            weights = param.view(-1)
            prob = torch.abs(weights) / torch.sum(torch.abs(weights))
            prob = prob + 1e-10  # Evitar log(0)
            total_entropy -= torch.sum(prob * torch.log(prob))
    return total_entropy

# Configurações de dados
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loop de treinamento com monitoramento da entropia
epochs = 50
lambda_entropy = 1  # Peso da regularização
entropies = []  # Lista para salvar entropias ao longo das épocas

for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    epoch_entropy = 0.5

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Regularização por entropia
        entropy_loss = entropy_regularization(model)
        total_loss = loss + lambda_entropy * entropy_loss

        # Backward pass e otimização
        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()
        epoch_entropy += entropy_loss.item()

    # Salvar entropia média da época
    epoch_entropy /= len(train_loader)
    entropies.append(epoch_entropy)

    # Avaliação no conjunto de teste
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)

    # Exibir métricas da época
    print(f"Epoch {epoch}: Train Loss = {running_loss / len(train_loader):.4f}, "
          f"Test Loss = {test_loss:.4f}, Accuracy = {accuracy:.2f}%, Entropy = {epoch_entropy:.4f}")

# Exibir evolução da entropia
print("\nEvolução da Entropia ao longo das épocas:")
for epoch, entropy in enumerate(entropies, 1):
    print(f"Epoch {epoch}: Entropy = {entropy:.4f}")


Epoch 1: Train Loss = 37.0150, Test Loss = 0.2120, Accuracy = 94.54%, Entropy = 36.5773
Epoch 2: Train Loss = 27.7074, Test Loss = 0.2224, Accuracy = 93.83%, Entropy = 27.4645
Epoch 3: Train Loss = 24.6936, Test Loss = 0.2456, Accuracy = 93.02%, Entropy = 24.4586
Epoch 4: Train Loss = 23.5434, Test Loss = 0.2491, Accuracy = 92.78%, Entropy = 23.2962
Epoch 5: Train Loss = 22.3529, Test Loss = 0.2705, Accuracy = 92.37%, Entropy = 22.0793
Epoch 6: Train Loss = 21.3779, Test Loss = 0.2597, Accuracy = 92.31%, Entropy = 21.0858
Epoch 7: Train Loss = 20.6839, Test Loss = 0.2533, Accuracy = 92.26%, Entropy = 20.4121
Epoch 8: Train Loss = 20.0685, Test Loss = 0.2307, Accuracy = 93.02%, Entropy = 19.8167
Epoch 9: Train Loss = 19.6857, Test Loss = 0.2236, Accuracy = 93.19%, Entropy = 19.4508
Epoch 10: Train Loss = 19.5199, Test Loss = 0.2062, Accuracy = 93.68%, Entropy = 19.2743
Epoch 11: Train Loss = 19.2240, Test Loss = 0.2255, Accuracy = 92.94%, Entropy = 18.9781
Epoch 12: Train Loss = 18.9138

In [29]:
import numpy as np

p_brasil = 0.5
p_espanha = 0.3
p_franca = 0.2

i_brasil = np.log2(1/p_brasil)
i_espanha = np.log2(1/p_espanha)
i_franca = np.log2(1/p_franca)

In [37]:
import numpy as np

# Dados para o histograma
data = np.random.randn(10)

counts, bin_edges = np.histogram(data, bins=10, density=True)

print("Densidade em cada intervalo:", counts)
print("Bordas dos intervalos:", bin_edges)


Densidade em cada intervalo: [0.26904525 0.5380905  0.5380905  0.         0.26904525 0.26904525
 0.26904525 0.26904525 0.         0.26904525]
Bordas dos intervalos: [-1.43764164 -1.06595695 -0.69427226 -0.32258757  0.04909711  0.4207818
  0.79246649  1.16415118  1.53583587  1.90752056  2.27920525]


In [38]:
data

array([-0.94924999,  2.27920525, -0.87306693, -1.43764164,  1.18779224,
       -0.54371496,  1.08192178,  0.62692954, -0.47510681,  0.23443902])

In [30]:
p_brasil * i_brasil + p_espanha * i_espanha + p_franca*i_franca

np.float64(1.4854752972273344)

In [22]:
i_brasil + i_espanha + i_franca

np.float64(5.058893689053568)

In [65]:
import torch.nn as nn
model = nn.Linear(3, 4)

size_model = 0
for param in model.parameters():
    size_model += len(param.view(-1)) * param.element_size()
size_model

64

In [61]:
12*32

384

In [53]:
64*4

256

In [ ]:
import torch.nn as nn
model = nn.Linear(3, 4)


def calculate_model_size(model, include_grad=False):
    size_model = 0
    for param in model.parameters():
        size_model += param.numel() * param.element_size()
        if include_grad:
            size_model += param.grad.numel() * param.grad.element_size()
    return size_model

calculate_model_size(model)